<a href="https://colab.research.google.com/github/ben854719/Retention-Engagement-Assistant-Smart-Reminders-for-Customer-Success/blob/main/backend_simple_reminder_streamlit_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st

Overwriting app.py


In [ ]:
!apt-get install -y nodejs npm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
npm is already the newest version (8.5.1~ds-1).
nodejs is already the newest version (12.22.9~dfsg-1ubuntu3.6).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!npm install express node-cron body-parser sqlite3

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
up to date, audited 182 packages in 2s
⠋
⠋32 packages are looking for funding
⠋  run `npm fund` for details
⠋
found 0 vulnerabilities
⠋

In [ ]:
%%writefile server.js
const express = require('express');
const cron = require('node-cron');
const bodyParser = require('body-parser');
const sqlite3 = require('sqlite3').verbose();

const app = express();
app.use(bodyParser.json());

// Database Setup (SQLite)

const db = new sqlite3.Database('./reminders.db');

// Initialize table if not exists
db.serialize(() => {
  db.run(`
    CREATE TABLE IF NOT EXISTS customers (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      name TEXT,
      email TEXT,
      renewal_date TEXT,
      milestone_date TEXT
    )
  `);
});

// Helper Functions.

function getUpcomingRenewals(callback) {
  const query = `
    SELECT * FROM customers
    WHERE DATE(renewal_date) <= DATE('now', '+30 days')
  `;
  db.all(query, [], (err, rows) => {
    if (err) {
      console.error(err);
      return;
    }
    callback(rows);
  });
}

function getMilestones(callback) {
  const query = `
    SELECT * FROM customers
    WHERE DATE(milestone_date) = DATE('now')
  `;
  db.all(query, [], (err, rows) => {
    if (err) {
      console.error(err);
      return;
    }
    callback(rows);
  });
}

//Reminder Logic.

function sendReminder(customer, type) {
  console.log(`Reminder: ${type.toUpperCase()} for ${customer.name} (${customer.email})`);
  // In a real system, you'd send an email/SMS here
}

// Renewal trigger: runs daily at midnight
cron.schedule('0 0 * * *', () => {
  console.log('Checking upcoming renewals...');
  getUpcomingRenewals(customers => {
    customers.forEach(c => sendReminder(c, 'renewal'));
  });
});

// Milestone trigger: runs daily at 8am
cron.schedule('0 8 * * *', () => {
  console.log('Checking milestones...');
  getMilestones(customers => {
    customers.forEach(c => sendReminder(c, 'milestone'));
  });
});

// REST API Gateway

// Fetch customer profile
app.get('/customers/:id', (req, res) => {
  const id = req.params.id;
  db.get('SELECT * FROM customers WHERE id = ?', [id], (err, row) => {
    if (err) {
      res.status(500).json({ error: err.message });
      return;
    }
    res.json(row);
  });
});

// Add new customer.
app.post('/customers', (req, res) => {
  const { name, email, renewal_date, milestone_date } = req.body;
  db.run(
    `INSERT INTO customers (name, email, renewal_date, milestone_date)
     VALUES (?, ?, ?, ?)`,
    [name, email, renewal_date, milestone_date],
    function (err) {
      if (err) {
        res.status(500).json({ error: err.message });
        return;
      }
      res.json({ id: this.lastID });
    }
  );
});

// Trigger manual reminder
app.post('/reminder', (req, res) => {
  const { customerId, type } = req.body;
  db.get('SELECT * FROM customers WHERE id = ?', [customerId], (err, row) => {
    if (err || !row) {
      res.status(404).json({ error: 'Customer not found' });
      return;
    }
    sendReminder(row, type);
    res.json({ status: 'Reminder sent' });
  });
});

// Server Start.

const PORT = process.env.PORT || 3000;
app.listen(PORT, () => {
  console.log(`Reminder system running on port ${PORT}`);
});

Overwriting server.js


In [ ]:
%%writefile reminder.js
const nodemailer = require('nodemailer');
const transporter = nodemailer.createTransport({
  service: 'gmail',
  auth: { user: 'nguyenben85@gmail.com', pass: 'Ottawa23!' }
}

function sendEmailReminder(customer, type) {
  const mailOptions = {
    from: 'nguyenben85@gmail.com',
    to: customer.email,
    subject: `Reminder: ${type.toUpperCase()}`,
    text: `Hello ${customer.name}, this is your ${type} reminder.`
  };
  transporter.sendMail(mailOptions, (err, info) => {
    if (err) console.error(err);
    else console.log(`Email sent: ${info.response}`);
  });
}

// Example call
sendEmailReminder({ name: "Ben", email: "nguyenben85@gmail.com" }, "renewal");

Overwriting reminder.js
